<a href="https://colab.research.google.com/github/aishwarya-r29/DPL-2025/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
import os
import glob
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import load_model

# ============================
# STEP 1: Load & Merge Data
# ============================
def load_trend_data(folder_path="/content/drive/MyDrive/Colab Notebooks/TrendDataset"):
    all_files = glob.glob(os.path.join(folder_path, "*.csv"))
    df_list = []

    for file in all_files:
        try:
            # Try utf-8 first
            df = pd.read_csv(file, encoding="utf-8", on_bad_lines="skip")
        except UnicodeDecodeError:
            # Fallback to latin1
            df = pd.read_csv(file, encoding="latin1", on_bad_lines="skip")

        # Standardize year column names (2000 [YR2000] → 2000)
        df.rename(columns=lambda x: str(x).split()[0] if "YR" in str(x) else x, inplace=True)
        df_list.append(df)

    # Concatenate all CSVs
    merged = pd.concat(df_list, axis=0, ignore_index=True)

    # Extract only numeric year columns
    year_cols = [c for c in merged.columns if str(c).isdigit()]
    merged = merged[year_cols].transpose()  # years as rows

    merged.index = merged.index.astype(int)
    merged = merged.sort_index()  # ensure chronological order

    # Convert '..' to NaN
    merged.replace('..', np.nan, inplace=True)

    return merged

# ============================
# STEP 2: Prepare Data
# ============================
def prepare_sequences(data, look_back=5):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back])
        y.append(data[i+look_back])
    return np.array(X), np.array(y)

# ============================
# STEP 3: Build LSTM Model
# ============================
def build_model(input_shape):
    model = Sequential([
        LSTM(64, activation='tanh', input_shape=input_shape, return_sequences=True),
        Dropout(0.2),
        LSTM(32, activation='tanh'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# ============================
# MAIN EXECUTION
# ============================
if __name__ == "__main__":
    # Load dataset
    data = load_trend_data("/content/drive/MyDrive/Colab Notebooks/TrendDataset")
    print("Data Shape:", data.shape)

    # --- Pick one indicator for univariate forecasting ---
    series = None
    chosen_col = None
    look_back = 5  # Define look_back here

    for col in data.columns:
        candidate = data[col].dropna()
        # Ensure candidate has enough data points to create at least one sequence
        if len(candidate) >= look_back + 1:
            series = candidate
            chosen_col = col # Store the chosen column index
            break

    if series is None:
        print(f"❌ No column found with at least {look_back + 1} non-NaN data points after cleaning.")
    else:
        print(f"✅ Using column with index: {chosen_col} and shape: {series.shape}")
        values = series.values.reshape(-1, 1)

        # Normalize
        scaler = MinMaxScaler()
        scaled = scaler.fit_transform(values)

        # Prepare sequences
        X, y = prepare_sequences(scaled, look_back=look_back)
        X = X.reshape((X.shape[0], X.shape[1], 1))  # [samples, timesteps, features]

        print("X Shape:", X.shape, "y Shape:", y.shape)

        # Build & train
        model = build_model((X.shape[1], X.shape[2]))
        model.fit(X, y, epochs=20, batch_size=16, verbose=1)

        # Save model in the native Keras format
        os.makedirs("model_files", exist_ok=True)
        model.save("model_files/Model.keras")
        print("✅ LSTM Model saved at model_files/Model.keras")

/tmp/ipython-input-2517528818.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged.replace('..', np.nan, inplace=True)


Data Shape: (25, 370237)
✅ Using column with index: 41938 and shape: (10,)
X Shape: (5, 5, 1) y Shape: (5, 1)
Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.6709
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.6319
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.5874
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.5454
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.5228
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.4903
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.4640
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.4189
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.3983
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.3625
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.3383
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.3021
Epoch 13/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.2863
Epoch 14/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.2767
Epoch 15/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.2399
Epoch 16/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/st

In [21]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the saved model in the native Keras format
model = load_model("model_files/Model.keras")

# Predict the next value after the last sequence
last_seq = scaled[-5:].reshape((1, 5, 1))  # last 5 timesteps
next_pred_scaled = model.predict(last_seq)

# Inverse scale back to original values
next_pred = scaler.inverse_transform(next_pred_scaled)
print("🔮 Next predicted value:", next_pred[0][0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step
🔮 Next predicted value: 13.919461
